<a href="https://colab.research.google.com/github/spatank/CIS-530/blob/master/Homework%205/homework_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# pip install pymagnitude
from pymagnitude import *

In [0]:
texts_path = '/Users/sppatankar/Developer/CIS-530/Homework 5/Data/data'
vectors_path = '/Users/sppatankar/Developer/CIS-530/Homework 5/Data/vectors'